In [3]:
#roda com ambiente densephrases
#!git clone -b v1.1.0 https://github.com/princeton-nlp/DensePhrases.git

# Train Question and Phrase Encoders

In [ ]:
# pip install -U pydantic spacy==3.4.4


In [8]:
#!pip install git+https://github.com/huggingface/transformers

### Load methods

In [1]:
import argparse
from arguments import parse_args
import json
import logging
import math
import os
import random
from pathlib import Path

import datasets
import evaluate
import numpy as np
import torch
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import set_seed
from datasets import load_dataset
from huggingface_hub import Repository, create_repo
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
# from utils_qa import postprocess_qa_predictions

import transformers
from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    get_scheduler,
)
#from transformers.utils import check_min_version, get_full_repo_name, send_example_telemetry
from transformers.utils.versions import require_version

/home/miu/miniconda3/envs/proconvqa/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
#check_min_version("4.27.0.dev0")

require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/question-answering/requirements.txt")

logger = get_logger(__name__)
# You should update this to your particular problem to have better documentation of `model_type`
MODEL_CONFIG_CLASSES = list(MODEL_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [3]:
args = parse_args()

# Sending telemetry. Tracking the example usage helps us better allocate resources to maintain them. The
# information sent is the one passed as arguments along with your Python/PyTorch versions.
#send_example_telemetry("run_qa_no_trainer", args)

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
# If we're using tracking, we also need to initialize it here and it will by default pick up all supported trackers
# in the environment
accelerator_log_kwargs = {}

if args.with_tracking:
    accelerator_log_kwargs["log_with"] = args.report_to
    accelerator_log_kwargs["logging_dir"] = args.output_dir

accelerator = Accelerator(gradient_accumulation_steps=args.gradient_accumulation_steps, **accelerator_log_kwargs)

# Make one log on every process with the configuration for debugging.
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state, main_process_only=False)
if accelerator.is_local_main_process:
    datasets.utils.logging.set_verbosity_warning()
    transformers.utils.logging.set_verbosity_info()
else:
    datasets.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_error()

# If passed along, set the training seed now.
if args.seed is not None:
    set_seed(args.seed)

# Handle the repository creation
if accelerator.is_main_process:
    if args.push_to_hub:
        if args.hub_model_id is None:
            repo_name = get_full_repo_name(Path(args.output_dir).name, token=args.hub_token)
        else:
            repo_name = args.hub_model_id
        create_repo(repo_name, exist_ok=True, token=args.hub_token)
        repo = Repository(args.output_dir, clone_from=repo_name, token=args.hub_token)

        with open(os.path.join(args.output_dir, ".gitignore"), "w+") as gitignore:
            if "step_*" not in gitignore:
                gitignore.write("step_*\n")
            if "epoch_*" not in gitignore:
                gitignore.write("epoch_*\n")
    elif args.output_dir is not None:
        os.makedirs(args.output_dir, exist_ok=True)
accelerator.wait_for_everyone()

12/29/2023 11:55:43 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no



In [4]:
# args.model_name_or_path='xlnet-base-cased'
#args.model_name_or_path='google/electra-small-discriminator'
# args.model_name_or_path='SpanBERT/spanbert-base-cased'
#distilbert
# args.model_name_or_path='distilbert-base-uncased'
args.model_name_or_path='bert-base-uncased'

if args.config_name:
    config = AutoConfig.from_pretrained(args.config_name)
elif args.model_name_or_path:
    config = AutoConfig.from_pretrained(args.model_name_or_path)
else:
    config = CONFIG_MAPPING[args.model_type]()
    logger.warning("You are instantiating a new config instance from scratch.")

if args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, use_fast=True)
elif args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=True)
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this script."
        "You can do it from another script, save it, and load it from here, using --tokenizer_name."
    )

https://huggingface.co/bert-base-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /home/miu/.var/app/com.visualstudio.code/cache/huggingface/transformers/tmp3dovd_hy
Downloading: 100%|██████████| 570/570 [00:00<00:00, 189kB/s]
storing https://huggingface.co/bert-base-uncased/resolve/main/config.json in cache at /home/miu/.var/app/com.visualstudio.code/cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
creating metadata file for /home/miu/.var/app/com.visualstudio.code/cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/miu/.var/app/com.visualstudio.code/cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51

In [5]:
import copy
from encoder import Encoder
from functools import partial
from transformers import AutoModel

# Prepare PLM if not load_dir
pretrained = None
phrase_only=False
query_only=False

load_encoder=False

if not load_encoder:
    pretrained = AutoModel.from_pretrained(
        args.model_name_or_path,
        config=config,
        cache_dir=None,
    )
    load_class = Encoder
    logger.info(f'DensePhrases encoder initialized with {args.model_name_or_path} ({pretrained.__class__})')
else:
    load_class = partial(
        Encoder.from_pretrained,
        pretrained_model_name_or_path=args.load_dir,
        cache_dir=args.cache_dir if args.cache_dir else None,
    )
    logger.info(f'DensePhrases encoder loaded from {args.load_dir}')

# DensePhrases encoder object
model = load_class(
    config=config,
    tokenizer=tokenizer,
    transformer_cls=MODEL_MAPPING[config.__class__],
    pretrained=copy.deepcopy(pretrained) if pretrained is not None else None,
    lambda_kl= 0.0,
    lambda_neg=0.1,
    lambda_flt=0.0,
    pbn_size=0.0,
    return_phrase=phrase_only,
    return_query=query_only,
)
    

https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin not found in cache or force_download set to True, downloading to /home/miu/.var/app/com.visualstudio.code/cache/huggingface/transformers/tmp7tum2o5u
Downloading: 100%|██████████| 420M/420M [00:14<00:00, 31.0MB/s] 
storing https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin in cache at /home/miu/.var/app/com.visualstudio.code/cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
creating metadata file for /home/miu/.var/app/com.visualstudio.code/cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache at /home/miu/.var/app/com.visualstudio.code/cache/huggingface/transformers/a8041bf617d7

### Tokenize data

In [15]:
from data_utils import prepare_train_features,prepare_validation_features
from data_utils import get_datasets
args.max_query_length=10
raw_datasets,dataset_infos=get_datasets(args,tokenizer,logger)
raw_datasets,dataset_infos

12/29/2023 01:25:25 - WARNING - datasets.builder - Found cached dataset squad (/home/miu/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 835.10it/s]


(DatasetDict({
     train: Dataset({
         features: ['id', 'title', 'context', 'question', 'answers'],
         num_rows: 87599
     })
     validation: Dataset({
         features: ['id', 'title', 'context', 'question', 'answers'],
         num_rows: 10570
     })
 }),
 {'question_column_name': 'question',
  'column_names': ['id', 'title', 'context', 'question', 'answers'],
  'context_column_name': 'context',
  'answer_column_name': 'answers',
  'pad_on_right': True,
  'max_seq_length': 384})

In [16]:
if "train" not in raw_datasets:
    raise ValueError("--do_train requires a train dataset")
train_dataset = raw_datasets["train"]
if args.max_train_samples is not None:
    # We will select sample from whole data if agument is specified
    train_dataset = train_dataset.select(range(args.max_train_samples))

# Create train feature from dataset
with accelerator.main_process_first():
    train_dataset = train_dataset.map(
        # prepare_train_features,
        lambda x: prepare_train_features(x, dataset_infos, tokenizer, args ),
        batched=True,
        num_proc=args.preprocessing_num_workers,
        remove_columns=dataset_infos['column_names'],
        load_from_cache_file=not args.overwrite_cache,
        desc="Running tokenizer on train dataset",
    )
    if args.max_train_samples is not None:
        # Number of samples might increase during Feature Creation, We select only specified max samples
        train_dataset = train_dataset.select(range(args.max_train_samples))

if "validation" not in raw_datasets:
    raise ValueError("--do_eval requires a validation dataset")
eval_examples = raw_datasets["validation"]
if args.max_eval_samples is not None:
    # We will select sample from whole data
    eval_examples = eval_examples.select(range(args.max_eval_samples))
# Validation Feature Creation
with accelerator.main_process_first():
    eval_dataset = eval_examples.map(
        # prepare_validation_features,
        lambda x: prepare_validation_features(x, dataset_infos=dataset_infos, tokenizer=tokenizer, args=args),
        batched=True,
        num_proc=args.preprocessing_num_workers,
        remove_columns=dataset_infos['column_names'],
        load_from_cache_file=not args.overwrite_cache,
        desc="Running tokenizer on validation dataset",
    )


Running tokenizer on train dataset:   0%|          | 0/88 [00:00<?, ?ba/s]

Running tokenizer on validation dataset:  91%|█████████ | 10/11 [00:02<00:00,  3.96ba/s]


In [9]:
from data_utils import get_dataloaders
args.per_device_train_batch_size=16
train_dataloader,eval_dataloader,predict_dataloader=get_dataloaders(args,tokenizer,raw_datasets,train_dataset,eval_dataset,dataset_infos,accelerator,logger)

12/28/2023 22:32:33 - INFO - __main__ - Sample 56952 of the training set: {'input_ids': [101, 2466, 1564, 1610, 27719, 1161, 1110, 1145, 1227, 1112, 1184, 136, 102, 5408, 1107, 1103, 3433, 6692, 1104, 2638, 1105, 2466, 1564, 1610, 27719, 1161, 117, 1137, 1103, 1822, 1583, 1105, 1146, 6754, 117, 4634, 1103, 1741, 117, 2670, 117, 1105, 1934, 1297, 1104, 1103, 1352, 1121, 1103, 4186, 1235, 1103, 3116, 1432, 119, 1103, 12600, 4669, 1107, 2638, 1564, 1610, 27719, 1161, 1108, 3035, 17118, 1118, 7162, 1121, 3738, 4035, 1403, 1931, 1105, 1103, 188, 11627, 2944, 27929, 119, 1103, 1146, 2528, 8355, 1616, 1104, 2466, 1564, 1610, 27719, 1161, 1108, 3035, 17118, 1118, 188, 18982, 1116, 118, 178, 4889, 1324, 117, 4035, 23655, 2737, 117, 1105, 176, 14170, 5641, 7418, 117, 1103, 1177, 118, 1270, 107, 1884, 19989, 107, 119, 7190, 1219, 1103, 2286, 118, 1106, 1523, 4186, 1432, 117, 1103, 188, 18982, 1116, 118, 178, 4889, 1324, 1121, 1184, 1110, 2052, 2350, 178, 9261, 5709, 1127, 1103, 2026, 1664, 118, 4

### Train model

In [10]:
import numpy as np
from tqdm.auto import tqdm
import logging
from model_utils import postprocess_qa_predictions,post_processing_function,create_and_fill_np_array
logger = logging.getLogger(__name__)


metric = evaluate.load("squad_v2" if args.version_2_with_negative else "squad")

In [11]:
# Optimizer
# Split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

# Scheduler and math around the number of training steps.
overrode_max_train_steps = False
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)
if args.max_train_steps is None:
    args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
    overrode_max_train_steps = True

lr_scheduler = get_scheduler(
    name=args.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=args.num_warmup_steps * args.gradient_accumulation_steps,
    num_training_steps=args.max_train_steps * args.gradient_accumulation_steps,
)

# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader, lr_scheduler
)

# We need to recalculate our total training steps as the size of the training dataloader may have changed.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)
if overrode_max_train_steps:
    args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
# Afterwards we recalculate our number of training epochs
args.num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

# Figure out how many steps we should save the Accelerator states
checkpointing_steps = args.checkpointing_steps
if checkpointing_steps is not None and checkpointing_steps.isdigit():
    checkpointing_steps = int(checkpointing_steps)

# We need to initialize the trackers we use, and also store our configuration.
# The trackers initializes automatically on the main process.
if args.with_tracking:
    experiment_config = vars(args)
    # TensorBoard cannot log Enums, need the raw value
    experiment_config["lr_scheduler_type"] = experiment_config["lr_scheduler_type"].value
    accelerator.init_trackers("qa_no_trainer", experiment_config)

In [12]:
# Train!
total_batch_size = args.per_device_train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {args.num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {args.per_device_train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {args.max_train_steps}")

# Only show the progress bar once on each machine.
progress_bar = tqdm(range(args.max_train_steps), disable=not accelerator.is_local_main_process)
completed_steps = 0
starting_epoch = 0

# Potentially load in the weights and states from a previous save
if args.resume_from_checkpoint:
    if args.resume_from_checkpoint is not None or args.resume_from_checkpoint != "":
        accelerator.print(f"Resumed from checkpoint: {args.resume_from_checkpoint}")
        accelerator.load_state(args.resume_from_checkpoint)
        path = os.path.basename(args.resume_from_checkpoint)
    else:
        # Get the most recent checkpoint
        dirs = [f.name for f in os.scandir(os.getcwd()) if f.is_dir()]
        dirs.sort(key=os.path.getctime)
        path = dirs[-1]  # Sorts folders by date modified, most recent checkpoint is the last
    # Extract `epoch_{i}` or `step_{i}`
    training_difference = os.path.splitext(path)[0]

    if "epoch" in training_difference:
        starting_epoch = int(training_difference.replace("epoch_", "")) + 1
        resume_step = None
    else:
        resume_step = int(training_difference.replace("step_", ""))
        starting_epoch = resume_step // len(train_dataloader)
        resume_step -= starting_epoch * len(train_dataloader)

12/28/2023 22:32:49 - INFO - __main__ - ***** Running training *****
12/28/2023 22:32:49 - INFO - __main__ -   Num examples = 89357
12/28/2023 22:32:49 - INFO - __main__ -   Num Epochs = 3
12/28/2023 22:32:49 - INFO - __main__ -   Instantaneous batch size per device = 16
12/28/2023 22:32:49 - INFO - __main__ -   Total train batch size (w. parallel, distributed & accumulation) = 16
12/28/2023 22:32:49 - INFO - __main__ -   Gradient Accumulation steps = 1
12/28/2023 22:32:49 - INFO - __main__ -   Total optimization steps = 16755
  0%|          | 0/16755 [00:00<?, ?it/s]

In [13]:
args.num_train_epochs=1

In [17]:
args.output_dir='/home/miu/Projects/NLP/DensePhrasesMinimal/result_models'

In [18]:
for epoch in range(starting_epoch, args.num_train_epochs):
    model.train()
    if args.with_tracking:
        total_loss = 0
    for step, batch in enumerate(train_dataloader):
        # We need to skip steps until we reach the resumed step
        if args.resume_from_checkpoint and epoch == starting_epoch:
            if resume_step is not None and step < resume_step:
                completed_steps += 1
                continue

        with accelerator.accumulate(model):
            outputs = model(**batch)
            loss=outputs[0]
            # loss = outputs.loss
            # We keep track of the loss at each epoch
            if args.with_tracking:
                total_loss += loss.detach().float()

            accelerator.backward(loss)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        # Checks if the accelerator has performed an optimization step behind the scenes
        if accelerator.sync_gradients:
            progress_bar.update(1)
            completed_steps += 1

        if isinstance(checkpointing_steps, int):
            if completed_steps % checkpointing_steps == 0:
                output_dir = f"step_{completed_steps }"
                if args.output_dir is not None:
                    output_dir = os.path.join(args.output_dir, output_dir)
                accelerator.save_state(output_dir)

        if completed_steps >= args.max_train_steps:
            break

    if args.checkpointing_steps == "epoch":
        output_dir = f"epoch_{epoch}"
        if args.output_dir is not None:
            output_dir = os.path.join(args.output_dir, output_dir)
        accelerator.save_state(output_dir)

    if epoch < args.num_train_epochs - 1:
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(
            args.output_dir, is_main_process=accelerator.is_main_process, save_function=accelerator.save
        )
        if accelerator.is_main_process:
            tokenizer.save_pretrained(args.output_dir)

 35%|███▍      | 5804/16755 [18:15<28:35,  6.38it/s]   

In [19]:
len(outputs)

5

In [28]:
outputs[1].shape

torch.Size([13, 341])

In [29]:
args.output_dir='/home/miu/Projects/NLP/results/'

In [30]:
# Evaluation
logger.info("***** Running Evaluation *****")
logger.info(f"  Num examples = {len(eval_dataset)}")
logger.info(f"  Batch size = {args.per_device_eval_batch_size}")

all_start_logits = []
all_end_logits = []

model.eval()

for step, batch in enumerate(eval_dataloader):
    with torch.no_grad():
        outputs = model(**batch)
        # start_logits = outputs.start_logits
        # end_logits = outputs.end_logits

        start_logits = outputs[0]
        end_logits = outputs[-1]

        if not args.pad_to_max_length:  # necessary to pad predictions and labels for being gathered
            start_logits = accelerator.pad_across_processes(start_logits, dim=1, pad_index=-100)
            end_logits = accelerator.pad_across_processes(end_logits, dim=1, pad_index=-100)

        all_start_logits.append(accelerator.gather_for_metrics(start_logits).cpu().numpy())
        all_end_logits.append(accelerator.gather_for_metrics(end_logits).cpu().numpy())

max_len = max([x.shape[1] for x in all_start_logits])  # Get the max_length of the tensor

# concatenate the numpy array
start_logits_concat = create_and_fill_np_array(all_start_logits, eval_dataset, max_len)
end_logits_concat = create_and_fill_np_array(all_end_logits, eval_dataset, max_len)

# delete the list of numpy arrays
del all_start_logits
del all_end_logits

outputs_numpy = (start_logits_concat, end_logits_concat)
prediction = post_processing_function(eval_examples, eval_dataset, outputs_numpy)
eval_metric = metric.compute(predictions=prediction.predictions, references=prediction.label_ids)
logger.info(f"Evaluation metrics: {eval_metric}")

# Prediction
if args.do_predict:
    logger.info("***** Running Prediction *****")
    logger.info(f"  Num examples = {len(predict_dataset)}")
    logger.info(f"  Batch size = {args.per_device_eval_batch_size}")

    all_start_logits = []
    all_end_logits = []

    model.eval()

    for step, batch in enumerate(predict_dataloader):
        with torch.no_grad():
            outputs = model(**batch)
            # start_logits = outputs.start_logits
            # end_logits = outputs.end_logits

            start_logits = outputs[0]
            end_logits = outputs[-1]

            if not args.pad_to_max_length:  # necessary to pad predictions and labels for being gathered
                start_logits = accelerator.pad_across_processes(start_logits, dim=1, pad_index=-100)
                end_logits = accelerator.pad_across_processes(end_logits, dim=1, pad_index=-100)

            all_start_logits.append(accelerator.gather_for_metrics(start_logits).cpu().numpy())
            all_end_logits.append(accelerator.gather_for_metrics(end_logits).cpu().numpy())

    max_len = max([x.shape[1] for x in all_start_logits])  # Get the max_length of the tensor
    # concatenate the numpy array
    start_logits_concat = create_and_fill_np_array(all_start_logits, predict_dataset, max_len)
    end_logits_concat = create_and_fill_np_array(all_end_logits, predict_dataset, max_len)

    # delete the list of numpy arrays
    del all_start_logits
    del all_end_logits

    outputs_numpy = (start_logits_concat, end_logits_concat)
    prediction = post_processing_function(predict_examples, predict_dataset, outputs_numpy)
    predict_metric = metric.compute(predictions=prediction.predictions, references=prediction.label_ids)
    logger.info(f"Predict metrics: {predict_metric}")

if args.with_tracking:
    log = {
        "squad_v2" if args.version_2_with_negative else "squad": eval_metric,
        "train_loss": total_loss.item() / len(train_dataloader),
        "epoch": epoch,
        "step": completed_steps,
    }
if args.do_predict:
    log["squad_v2_predict" if args.version_2_with_negative else "squad_predict"] = predict_metric

    accelerator.log(log, step=completed_steps)

if args.output_dir is not None:
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(
        args.output_dir, is_main_process=accelerator.is_main_process, save_function=accelerator.save
    )
    if accelerator.is_main_process:
        tokenizer.save_pretrained(args.output_dir)
        if args.push_to_hub:
            repo.push_to_hub(commit_message="End of training", auto_lfs_prune=True)

        logger.info(json.dumps(eval_metric, indent=4))
        save_prefixed_metrics(eval_metric, args.output_dir)

12/27/2023 16:43:46 - INFO - __main__ - ***** Running Evaluation *****
12/27/2023 16:43:46 - INFO - __main__ -   Num examples = 10895
12/27/2023 16:43:46 - INFO - __main__ -   Batch size = 8
100%|██████████| 10570/10570 [00:08<00:00, 1270.13it/s]
Configuration saved in /home/miu/Projects/NLP/PRO-ConvQA/results/config.json
Model weights saved in /home/miu/Projects/NLP/PRO-ConvQA/results/pytorch_model.bin
tokenizer config file saved in /home/miu/Projects/NLP/PRO-ConvQA/results/tokenizer_config.json
Special tokens file saved in /home/miu/Projects/NLP/PRO-ConvQA/results/special_tokens_map.json


# Inference

### Load methods and model

In [21]:
""" Finetuning the library models for question-answering on SQuAD (DistilBERT, Bert, XLM, XLNet)."""

import timeit
import h5py
import argparse
from arguments import parse_args
import json
import logging
import math
import os
import random
from pathlib import Path

import datasets
import evaluate
import numpy as np
import torch
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import set_seed
from datasets import load_dataset
from huggingface_hub import Repository, create_repo
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
# from utils_qa import postprocess_qa_predictions

import transformers
from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    SchedulerType,
    default_data_collator,
    get_scheduler,
)
#from transformers.utils import check_min_version, get_full_repo_name, send_example_telemetry
from transformers.utils.versions import require_version


logger = logging.getLogger(__name__)

In [22]:
args = parse_args()

# Sending telemetry. Tracking the example usage helps us better allocate resources to maintain them. The
# information sent is the one passed as arguments along with your Python/PyTorch versions.
#send_example_telemetry("run_qa_no_trainer", args)

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
# If we're using tracking, we also need to initialize it here and it will by default pick up all supported trackers
# in the environment
accelerator_log_kwargs = {}

if args.with_tracking:
    accelerator_log_kwargs["log_with"] = args.report_to
    accelerator_log_kwargs["logging_dir"] = args.output_dir

accelerator = Accelerator(gradient_accumulation_steps=args.gradient_accumulation_steps, **accelerator_log_kwargs)

# Make one log on every process with the configuration for debugging.
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
#logger.info(accelerator.state, main_process_only=False)
if accelerator.is_local_main_process:
    datasets.utils.logging.set_verbosity_warning()
    transformers.utils.logging.set_verbosity_info()
else:
    datasets.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_error()

# If passed along, set the training seed now.
if args.seed is not None:
    set_seed(args.seed)

# Handle the repository creation
if accelerator.is_main_process:
    if args.push_to_hub:
        if args.hub_model_id is None:
            repo_name = get_full_repo_name(Path(args.output_dir).name, token=args.hub_token)
        else:
            repo_name = args.hub_model_id
        create_repo(repo_name, exist_ok=True, token=args.hub_token)
        repo = Repository(args.output_dir, clone_from=repo_name, token=args.hub_token)

        with open(os.path.join(args.output_dir, ".gitignore"), "w+") as gitignore:
            if "step_*" not in gitignore:
                gitignore.write("step_*\n")
            if "epoch_*" not in gitignore:
                gitignore.write("epoch_*\n")
    elif args.output_dir is not None:
        os.makedirs(args.output_dir, exist_ok=True)
accelerator.wait_for_everyone()

In [23]:
import copy
from encoder import Encoder
from functools import partial
from transformers import AutoModel

def load_encoder(device, args, phrase_only=False, query_only=False, freeze_embedding=True):
    # Distributed training:
    # The .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.
    config = AutoConfig.from_pretrained(
        args.config_name if args.config_name else args.pretrained_name_or_path,
        cache_dir=args.cache_dir if args.cache_dir else None,
    )
    tokenizer = AutoTokenizer.from_pretrained(
        args.tokenizer_name if args.tokenizer_name else args.pretrained_name_or_path,
        cache_dir=args.cache_dir if args.cache_dir else None,
        use_fast=True,
    )

    # Prepare PLM if not load_dir
    pretrained = None
    load_class = partial(
        Encoder.from_pretrained,
        pretrained_model_name_or_path=args.load_dir,
        cache_dir=args.cache_dir if args.cache_dir else None,
    )
    logger.info(f'DensePhrases encoder loaded from {args.load_dir}')

    # DensePhrases encoder object
    model = load_class(
        config=config,
        tokenizer=tokenizer,
        transformer_cls=MODEL_MAPPING[config.__class__],
        pretrained=copy.deepcopy(pretrained) if pretrained is not None else None,
        lambda_kl=getattr(args, 'lambda_kl', 0.0),
        lambda_neg=getattr(args, 'lambda_neg', 0.0),
        lambda_flt=getattr(args, 'lambda_flt', 0.0),
        pbn_size=getattr(args, 'pbn_size', 0.0),
        return_phrase=phrase_only,
        return_query=query_only,
    )
    # Phrase only (for phrase embedding)
    if phrase_only:
        if hasattr(model, "module"):
            del model.module.query_start_encoder
            del model.module.query_end_encoder
        else:
            del model.query_start_encoder
            del model.query_end_encoder
        logger.info("Load only phrase encoders for embedding phrases")
    
    
    if freeze_embedding:
        for name, param in model.named_parameters():
            if name.endswith(".embeddings.word_embeddings.weight"):
                param.requires_grad = False
                logger.info(f'freezing {name}')

    model.to(device)
    logger.info('Number of model parameters: {:,}'.format(sum(p.numel() for p in model.parameters())))
    return model, tokenizer, config

In [24]:

args.pretrained_name_or_path='results'
args.load_dir='results'
args.cache_dir=None
args.tokenizer_name='SpanBERT/spanbert-base-cased'
args.config_name='SpanBERT/spanbert-base-cased'
"""
args.pretrained_name_or_path='princeton-nlp/densephrases-multi'
args.load_dir='princeton-nlp/densephrases-multi'
args.cache_dir=None
args.tokenizer_name='SpanBERT/spanbert-base-cased'
args.config_name='SpanBERT/spanbert-base-cased'
"""

args.max_query_length=50

model, tokenizer, _ = load_encoder(accelerator.device, args, phrase_only=False)

loading configuration file https://huggingface.co/SpanBERT/spanbert-base-cased/resolve/main/config.json from cache at /home/miu/.var/app/com.visualstudio.code/cache/huggingface/transformers/db038c967205352871fe5924ad8dd767343ca3c4869744e2e81eae6c199e8721.5b1ff81c06025be8bc48ae0cf34bb012d6cbc7f8918e8bcd354f80e6952b3ae7
Model config BertConfig {
  "_name_or_path": "SpanBERT/spanbert-base-cased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.13.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

Could not locate the tokenizer configuration file, will try to 

In [5]:
#!pip install faiss-gpu==1.6.5 blosc==1.10.6 spacy ujson transformers==4.13.0

In [6]:
#!mv /home/kiki/dados_servidor/Servidor/InformationRetrival/densephrases/DensePhrases/scripts .

In [25]:

from transformers.data.data_collator import default_data_collator, torch_default_data_collator
from datasets import load_dataset


from densephrases.utils.trainer_qa import QuestionAnsweringTrainer
from transformers import (
    DataCollatorWithPadding,
    default_data_collator,
)
from densephrases.utils.single_utils import set_seed, load_encoder
from densephrases import Options
from scripts.preprocess.convert_squad_to_hf import convert_squad_to_hf

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/miu/miniconda3/envs/proconvqa/lib/python3.8/site-packages/thinc/compat.py:36: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  hasattr(torch, "has_mps")
/home/miu/miniconda3/envs/proconvqa/lib/python3.8/site-packages/thinc/compat.py:37: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  and torch.has_mps  # type: ignore[attr-defined]


In [26]:
data_files = {}
data_files["test"] = convert_squad_to_hf('/home/miu/Projects/NLP/PRO-ConvQA/wikidump/wiki-dev/0000')

Saving 18829 (context, question, answer) triples.
Writing to /home/miu/Projects/NLP/PRO-ConvQA/wikidump/wiki-dev/0000_hf.json



### Load and tokenize data

In [27]:
# Create phrase vectors
args.draft = False
args.pad_to_max_length=True
args.append_title=False
args.convert_squad_to_hf=True
args.remove_unused_columns=True
args.use_legacy_prediction_loop=False
args.world_size=1
args.eval_batch_size=8
args.dataloader_drop_last=True
args.dataloader_pin_memory=True
args.output_dir='output_test/'
args.test_file='/home/miu/Projects/NLP/PRO-ConvQA/wikidump/wiki-dev/0000'
args.prediction_loss_only=False
args.n_gpu=1
args.past_index=1

In [28]:
filter_only=False

output_path = 'dump/phrase' if not filter_only else 'dump/filter'
if not os.path.exists(os.path.join(args.output_dir, output_path)):
    os.makedirs(os.path.join(args.output_dir, output_path))

start_time = timeit.default_timer()
if ':' not in args.test_file:
    test_files = [args.test_file]
    offsets = [0]
    output_dump_file = os.path.join(
        args.output_dir, f"{output_path}/{os.path.splitext(os.path.basename(args.test_file))[0]}.hdf5"
    )
else:
    dirname = os.path.dirname(args.test_file)
    basename = os.path.basename(args.test_file)
    start, end = list(map(int, basename.split(':')))
    output_dump_file = os.path.join(
        args.output_dir, f"{output_path}/{start}-{end}.hdf5"
    )

    # skip files if possible
    if os.path.exists(output_dump_file):
        with h5py.File(output_dump_file, 'r') as f:
            dids = list(map(int, f.keys()))
        start = int(max(dids) / 1000)
        logger.info('%s exists; starting from %d' % (output_dump_file, start))

    names = [str(i).zfill(4) for i in range(start, end)]
    test_files = [os.path.join(dirname, name) for name in names]
    offsets = [int(each) * 1000 for each in names]

for offset, test_file in zip(offsets, test_files):
    logger.info(f"***** Pre-processing contexts from {test_file} *****")

    data_files = {}
    if args.convert_squad_to_hf:
        data_files["test"] = convert_squad_to_hf(test_file)
    else:
        data_files["test"] = test_file
    extension = data_files["test"].split(".")[-1]
    raw_datasets = load_dataset(extension, data_files=data_files, field="data", cache_dir=args.cache_dir)

    column_names = raw_datasets["test"].column_names
    context_column_name = "context" if "context" in column_names else column_names[1]

    # Padding side determines if we do (question|context) or (context|question).
    pad_on_right = tokenizer.padding_side == "right"

    if args.max_seq_length > tokenizer.model_max_length:
        args.warning(
            f"The max_seq_length passed ({args.max_seq_length}) is larger than the maximum length for the"
            f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}."
        )
    max_seq_length = min(args.max_seq_length, tokenizer.model_max_length)

    # Validation preprocessing
    def prepare_validation_features(examples, indexes):
        # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
        # in one example possible giving several features when a context is long, each of those features having a
        # context that overlaps a bit the context of the previous feature.
        if args.append_title:
            tokenized_examples = tokenizer(
                examples['title' if pad_on_right else context_column_name],
                examples[context_column_name if pad_on_right else 'title'],
                truncation="only_second" if pad_on_right else "only_first",
                max_length=max_seq_length,
                stride=args.doc_stride,
                return_overflowing_tokens=True,
                return_offsets_mapping=True,
                padding="max_length" if args.pad_to_max_length else False,
            )
        else:
            tokenized_examples = tokenizer(
                examples[context_column_name],
                truncation="only_first",
                max_length=max_seq_length,
                stride=args.doc_stride,
                return_overflowing_tokens=True,
                return_offsets_mapping=True,
                padding="max_length" if args.pad_to_max_length else False,
            )

        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples["overflow_to_sample_mapping"]
        
        # Inflate doc_idxs based on sample_mapping
        tokenized_examples['doc_idx'] = [offset + examples['doc_idx'][i] for i in sample_mapping]

        # This example_id indicates the index of an original paragraph (not question id)
        tokenized_examples['example_id'] = [indexes[i] for i in sample_mapping]

        for i in range(len(tokenized_examples["input_ids"])):
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)
            context_index = 1 if pad_on_right and args.append_title else 0

            # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
            # position is part of the context or not.
            tokenized_examples["offset_mapping"][i] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(tokenized_examples["offset_mapping"][i])
            ]

        return tokenized_examples
    
    examples = raw_datasets["test"]

    # Predict Feature Creation
    with accelerator.main_process_first():
        dataset = examples.map(
            prepare_validation_features,
            batched=True,
            num_proc=args.preprocessing_num_workers,
            # num_proc=1,
            with_indices=True,
            remove_columns=column_names,
            load_from_cache_file=not args.overwrite_cache,
            desc="Running tokenizer on prediction dataset",
        )
    
    # Data collator
    # We have already padded to max length if the corresponding flag is True, otherwise we need to pad in the data
    # collator.
    data_collator = (
        default_data_collator
        if args.pad_to_max_length
        else DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8 if args.fp16 else None)
    )

Saving 18829 (context, question, answer) triples.
Writing to /home/miu/Projects/NLP/PRO-ConvQA/wikidump/wiki-dev/0000_hf.json



12/27/2023 19:27:50 - WARNING - datasets.builder - Using custom data configuration default-bdb26076fec24993


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1536.38it/s]


Dataset json downloaded and prepared to /home/miu/.var/app/com.visualstudio.code/cache/huggingface/datasets/json/default-bdb26076fec24993/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


Running tokenizer on prediction dataset:  95%|█████████▍| 18/19 [00:03<00:00,  5.12ba/s]


In [29]:
args.per_device_train_batch_size=8

predict_dataset_for_model = dataset.remove_columns([ "offset_mapping","doc_idx","overflow_to_sample_mapping"])
predict_dataloader = DataLoader(
    predict_dataset_for_model, shuffle=True, collate_fn=data_collator, batch_size=args.per_device_train_batch_size,
)

model, predict_dataloader = accelerator.prepare(model, predict_dataloader)

In [30]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy()

c=0
feature_id=0
all_predictions=[]
model.return_phrase=True
for step, inputs in enumerate(predict_dataloader):
    outputs=model(**inputs)

    for item_idx, (start, end, filter_start, filter_end) in enumerate(zip(*outputs)):
        a= {'example_id': inputs['example_id'][item_idx].item(),
            'feature_id': 0, # dataloader should not be shuffled
            'start': to_numpy(start),
            'end': to_numpy(end),
            'filter_start': to_numpy(filter_start),
            'filter_end': to_numpy(filter_end),
        }
        all_predictions.append(a)
    feature_id+=1
    c+=1
    if c>10:
        break

In [31]:
outputs[0].shape,outputs[1].shape,outputs[2].shape,outputs[3].shape

(torch.Size([8, 384, 768]),
 torch.Size([8, 384, 768]),
 torch.Size([8, 384]),
 torch.Size([8, 384]))

### Search Engine

In [32]:
inputs['input_ids'].shape

torch.Size([8, 384])

In [33]:
tokenizer.decode(inputs['input_ids'][0])

'[CLS] although the schedule for most of the personnel involved was grueling, the actors found the remote location for " part iii " relaxing, compared to shooting its predecessor. the role of clara clayton was written with mary steenburgen in mind. when she received the script, however, she was reluctant to commit to the film until her kids, who loved the original,\'hounded\'her. lloyd shared his first on - screen kiss with steenburgen in " part iii ". the hill valley festival dance scene proved to be the most dangerous for lloyd and steenburgen ; overzealous dancing left steenburgen with a torn ligament in her foot. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [34]:
df=[]
query = 'tell me about the historical spending on defense'

for i in range(1):
    df.append({'doc_idx':0, 'par_idx':0, 'title':'', 'context':query})
p='/home/miu/Projects/NLP/PRO-ConvQA/query.json'
with open(p, 'w') as f:
        json.dump({'data': df}, f)
r_d = load_dataset(extension, data_files=p, field="data", cache_dir=args.cache_dir)
r_d=r_d['train']
# query_inputs = prepare_features(r_d['train'])
max_seq_length=10
with accelerator.main_process_first():
    query_dataset = r_d.map(
        prepare_validation_features,
        batched=True,
        num_proc=1,
        with_indices=True,
        desc="Running tokenizer on prediction dataset",
    )
query_dataset = query_dataset.remove_columns(["par_idx", "offset_mapping","doc_idx","overflow_to_sample_mapping"])
query_dataloader = DataLoader(
    query_dataset, shuffle=True, collate_fn=data_collator, batch_size=1,
)
query_dataloader = accelerator.prepare(query_dataloader)

12/27/2023 19:28:01 - WARNING - datasets.builder - Using custom data configuration default-a5a6aeab93cc2962


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 2542.00it/s]


Dataset json downloaded and prepared to /home/miu/.var/app/com.visualstudio.code/cache/huggingface/datasets/json/default-a5a6aeab93cc2962/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


Running tokenizer on prediction dataset:   0%|          | 0/1 [00:00<?, ?ba/s]


In [35]:
model.return_query=True
model.return_phrase=False

for step, query_inputs in enumerate(query_dataloader):
    #print(query_inputs)
    query_outputs=model(input_ids_=query_inputs['input_ids'],attention_mask_=query_inputs['attention_mask'])
    
    break

In [36]:
q_start=query_outputs[0][0].cpu().detach().numpy()[0]
q_end=query_outputs[1][0].cpu().detach().numpy()[0]
q_start.shape,q_end.shape

((768,), (768,))

In [37]:
def search_mips(phrases_embs,query_emb):
    print(phrases_embs.shape)
    print(query_emb.shape)
    scores=[]
    for i in range(phrases_embs.shape[0]):
        batch=[]
        for j in range(phrases_embs.shape[1]):
            if j%10==0:
                #batch.append(phrases_embs[i][j])
                s=np.dot(phrases_embs[i][j],query_emb)
                scores.append((i,j,s))

        #reduced_embs.append(batch)
    scores=np.asarray(scores)
    return scores

scores=search_mips(outputs[0].cpu().detach().numpy(),q_start)

(8, 384, 768)
(768,)


In [38]:
best_score = np.argmax(scores[:,2])
best_score,scores[best_score]

(39, array([1.        , 0.        , 1.79579234]))

In [39]:
i=int(scores[best_score][0])
j=int(scores[best_score][1])
tokenizer.decode(inputs['input_ids'][i][j:])

'[CLS] symbiotic relationship between joker and batman, and sees the villain shatter the trust between batman and his adopted family. capullo\'s joker design replaced his traditional outfit with a utilitarian, messy, and disheveled appearance to convey that the character was on a mission ; his face ( surgically removed in 2011\'s " detective comics " # 1 ) was reattached with belts, wires, and hooks, and he was outfitted with mechanics overalls. the joker\'s face was restored in snyder\'s and capullo\'s " " ( 2014 ), the concluding chapter to " death of the family ". [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

### test with question answer module

In [9]:
def test():
    return 1

In [10]:
!pip install tensorboardX

In [11]:
args.fp16=False
args.bf16=False
data_collator = (
            default_data_collator
            if args.pad_to_max_length
            else DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8 if args.fp16 else None)
        )
args.seed=1000
args.skip_memory_metrics=True
#logger = logging.get_logger(__name__)
#log_levels = logging.get_log_levels_dict().copy()
#trainer_log_levels = dict(**log_levels, passive=-1)
args.get_process_log_level=test
args._setup_devices=False
args.sharded_ddp=[]
args.place_model_on_device=True
args.deepspeed=False
args.fp16_full_eval=False
args.bf16_full_eval=False
args.parallel_mode=False
args.disable_tqdm=False
args.should_save=False
args.max_steps=1000
args.half_precision_backend=False
args.device=torch.device('cuda')
args.report_to=['tensorboard']
args.label_smoothing_factor=0
args.label_names=None
trainer = QuestionAnsweringTrainer(
            model=model,
            args=args,
            tokenizer=tokenizer,
            data_collator=data_collator,
        )

max_steps is given, it will override any value given in num_train_epochs


In [13]:
# trainer.generate_phrase_vecs(dataset, examples, output_dump_file, offset, args)

In [15]:
def dump_phrases(args, model, tokenizer, filter_only=False):
    output_path = 'dump/phrase' if not filter_only else 'dump/filter'
    if not os.path.exists(os.path.join(args.output_dir, output_path)):
        os.makedirs(os.path.join(args.output_dir, output_path))

    start_time = timeit.default_timer()
    if ':' not in args.test_file:
        test_files = [args.test_file]
        offsets = [0]
        output_dump_file = os.path.join(
            args.output_dir, f"{output_path}/{os.path.splitext(os.path.basename(args.test_file))[0]}.hdf5"
        )
    else:
        dirname = os.path.dirname(args.test_file)
        basename = os.path.basename(args.test_file)
        start, end = list(map(int, basename.split(':')))
        output_dump_file = os.path.join(
            args.output_dir, f"{output_path}/{start}-{end}.hdf5"
        )

        # skip files if possible
        if os.path.exists(output_dump_file):
            with h5py.File(output_dump_file, 'r') as f:
                dids = list(map(int, f.keys()))
            start = int(max(dids) / 1000)
            logger.info('%s exists; starting from %d' % (output_dump_file, start))

        names = [str(i).zfill(4) for i in range(start, end)]
        test_files = [os.path.join(dirname, name) for name in names]
        offsets = [int(each) * 1000 for each in names]

    for offset, test_file in zip(offsets, test_files):
        logger.info(f"***** Pre-processing contexts from {test_file} *****")

        data_files = {}
        if args.convert_squad_to_hf:
            data_files["test"] = convert_squad_to_hf(test_file)
        else:
            data_files["test"] = test_file
        extension = data_files["test"].split(".")[-1]
        raw_datasets = load_dataset(extension, data_files=data_files, field="data", cache_dir=args.cache_dir)

        column_names = raw_datasets["test"].column_names
        context_column_name = "context" if "context" in column_names else column_names[1]

        # Padding side determines if we do (question|context) or (context|question).
        pad_on_right = tokenizer.padding_side == "right"

        if args.max_seq_length > tokenizer.model_max_length:
            args.warning(
                f"The max_seq_length passed ({args.max_seq_length}) is larger than the maximum length for the"
                f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}."
            )
        max_seq_length = min(args.max_seq_length, tokenizer.model_max_length)

        # Validation preprocessing
        def prepare_validation_features(examples, indexes):
            # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
            # in one example possible giving several features when a context is long, each of those features having a
            # context that overlaps a bit the context of the previous feature.
            if args.append_title:
                tokenized_examples = tokenizer(
                    examples['title' if pad_on_right else context_column_name],
                    examples[context_column_name if pad_on_right else 'title'],
                    truncation="only_second" if pad_on_right else "only_first",
                    max_length=max_seq_length,
                    stride=args.doc_stride,
                    return_overflowing_tokens=True,
                    return_offsets_mapping=True,
                    padding="max_length" if args.pad_to_max_length else False,
                )
            else:
                tokenized_examples = tokenizer(
                    examples[context_column_name],
                    truncation="only_first",
                    max_length=max_seq_length,
                    stride=args.doc_stride,
                    return_overflowing_tokens=True,
                    return_offsets_mapping=True,
                    padding="max_length" if args.pad_to_max_length else False,
                )

            # Since one example might give us several features if it has a long context, we need a map from a feature to
            # its corresponding example. This key gives us just that.
            sample_mapping = tokenized_examples["overflow_to_sample_mapping"]
            
            # Inflate doc_idxs based on sample_mapping
            tokenized_examples['doc_idx'] = [offset + examples['doc_idx'][i] for i in sample_mapping]

            # This example_id indicates the index of an original paragraph (not question id)
            tokenized_examples['example_id'] = [indexes[i] for i in sample_mapping]

            for i in range(len(tokenized_examples["input_ids"])):
                # Grab the sequence corresponding to that example (to know what is the context and what is the question).
                sequence_ids = tokenized_examples.sequence_ids(i)
                context_index = 1 if pad_on_right and args.append_title else 0

                # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
                # position is part of the context or not.
                tokenized_examples["offset_mapping"][i] = [
                    (o if sequence_ids[k] == context_index else None)
                    for k, o in enumerate(tokenized_examples["offset_mapping"][i])
                ]

            return tokenized_examples
        
        examples = raw_datasets["test"]

        # Predict Feature Creation
        with accelerator.main_process_first():
            dataset = examples.map(
                prepare_validation_features,
                batched=True,
                num_proc=args.preprocessing_num_workers,
                # num_proc=1,
                with_indices=True,
                remove_columns=column_names,
                load_from_cache_file=not args.overwrite_cache,
                desc="Running tokenizer on prediction dataset",
            )
        
        # Data collator
        # We have already padded to max length if the corresponding flag is True, otherwise we need to pad in the data
        # collator.
        data_collator = (
            default_data_collator
            if args.pad_to_max_length
            else DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8 if args.fp16 else None)
        )

        # Use trainer for predict
        trainer = QuestionAnsweringTrainer(
            model=model,
            args=args,
            tokenizer=tokenizer,
            data_collator=data_collator,
        )
        trainer.generate_phrase_vecs(dataset, examples, output_dump_file, offset, args)

        evalTime = timeit.default_timer() - start_time
        logger.info("Evaluation done in total %f secs (%f sec per example)", evalTime, evalTime / len(dataset))

In [19]:
# Create phrase vectors
args.draft = False
args.pad_to_max_length=True
args.append_title=False
args.convert_squad_to_hf=True
args.remove_unused_columns=True
args.use_legacy_prediction_loop=False
args.world_size=1
args.eval_batch_size=8
args.dataloader_drop_last=True
args.dataloader_pin_memory=True
args.output_dir='/home/kiki/dados_servidor/Servidor/InformationRetrival/densephrases/output/'
args.test_file='/home/kiki/dados_servidor/Servidor/InformationRetrival/densephrases/wikidump/wiki-dev/0000'
args.prediction_loss_only=False
args.n_gpu=1
args.past_index=1
dump_phrases(args, model, tokenizer, filter_only=False)

Saving 18829 (context, question, answer) triples.
Writing to /home/kiki/dados_servidor/Servidor/InformationRetrival/densephrases/wikidump/wiki-dev/0000_hf.json



Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 2137.77it/s]


Dataset json downloaded and prepared to /home/kiki/dados_servidor/Servidor/InformationRetrival/densephrases/results/json/default-f6398ca9aaf53f37/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 938.95it/s]
max_steps is given, it will override any value given in num_train_epochs                               
The following columns in the test set  don't have a corresponding argument in `Encoder.forward` and have been ignored: doc_idx, offset_mapping, overflow_to_sample_mapping.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Exception in thread Thread-106:
Traceback (most recent call last):
  File "/home/kiki/miniconda3/envs/densephrases/lib/python3.9/threading.py", line 950, in _bootstrap_inner
Exception in thread Thread-107:
Traceback (most recent call last):
  File "/home/kiki/miniconda3/envs/densephrases/lib/python3.9/threading.py", line 950, in _bootstrap_inner
Exception in thread Thread-108:
Traceback (most recent call last):
  File "/home/kiki/miniconda3/envs/densephrases/lib/python3.9/threading.py", line 950, in _bootstrap_inner
    self.run()
  File "/home/kiki/miniconda3/envs/densephrases/lib/python3.9/threading.py", line 888, in run
    self.run()
  File "/home/kiki/miniconda3/envs/densephrases/lib/python3.9/threading.py", line 888, in run
    self.run()
  File "/home/kiki/miniconda3/envs/densephrases/lib/python3.9/threading.py", line 888, in run
Exception in thread Thread-109:
Traceback (most recent call last):
  File "/home/kiki/miniconda3/envs/densephrases/lib/python3.9/threading.py", line 95

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


    fid = h5f.open(name, h5f.ACC_RDWR, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 106, in h5py.h5f.open
OSError: Unable to open file (truncated file: eof = 96, sblock->base_addr = 0, stored_eof = 2048)
Exception in thread Thread-116:
Traceback (most recent call last):
  File "/home/kiki/miniconda3/envs/densephrases/lib/python3.9/threading.py", line 950, in _bootstrap_inner
    self.run()
  File "/home/kiki/miniconda3/envs/densephrases/lib/python3.9/threading.py", line 888, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kiki/dados_servidor/Servidor/InformationRetrival/densephrases/densephrases/utils/embed_utils.py", line 198, in write
    with h5py.File(output_dump_file, 'a') as f:
  File "/home/kiki/miniconda3/envs/densephrases/lib/python3.9/site-packages/h5py/_hl/files.py", line 567, in __init__
Exception in thread Thread-117:


AttributeError: 'QuestionAnsweringTrainer' object has no attribute '_past'

OSError: Unable to open file (truncated file: eof = 96, sblock->base_addr = 0, stored_eof = 2048)
